In [ ]:
import numpy as np
from os import getcwd as cwd
from os.path import join as pj
import pandas as pd
from sklearn.model_selection import KFold
import torch
import torch.nn as nn
import torch.utils.data as data
import visdom

# Logger
from IO.logger import Logger
# model
from model.optimizer import AdamW

In [ ]:
class args:
    experiment_name = "linearNet_b100_l4_f100"
    # paths
    bbox_data_path = pj(cwd(), "data/bbox_data", "target_only_20200806.csv")
    model_root = pj(cwd(), "output_model/bbox2size", experiment_name)
    # model config
    linear_num = 4
    feature_num = 100
    # train config
    bs = 100
    lr = 1e-3
    nepoch = 1000
    # visdom
    visdom = True
    port = 8098

In [ ]:
if torch.cuda.is_available():
    args.cuda = True
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
else:
    args.cuda = False
    torch.set_default_tensor_type('torch.FloatTensor')
torch.multiprocessing.set_start_method('spawn')

#### Save args

In [ ]:
args_logger = Logger(args)
args_logger.save()

### visdom

In [ ]:
if args.visdom:
    # Create visdom
    vis = visdom.Visdom(port=args.port)
    
    win_train_loss = vis.line(
        X=np.array([0]),
        Y=np.array([0]),
        opts=dict(
            title='train_loss',
            xlabel='epoch',
            ylabel='loss',
            width=800,
            height=400
        )
    )
    win_test_loss = vis.line(
        X=np.array([0]),
        Y=np.array([0]),
        opts=dict(
            title='test_loss',
            xlabel='epoch',
            ylabel='loss',
            width=800,
            height=400
        )
    )

In [ ]:
def visualize(vis, phase, visualized_data, window):
    vis.line(
        X=np.array([phase]),
        Y=np.array([visualized_data]),
        update='append',
        win=window
    )

### dataset

In [ ]:
class bbox2size_dataset(data.Dataset):
    
    def __init__(self, bbox_data, size_data=None, training=False, evaluation=False):
        self.bbox_data = bbox_data
        self.size_data = size_data
        self.training = training
        self.evaluation = evaluation
        
    def __getitem__(self, index):
        bbox = self.bbox_data[index].astype("float32")
        bbox = torch.from_numpy(bbox)
        
        if self.training is True or self.evaluation is True:
            size = self.size_data[index].astype("float32")
            return bbox, size
        else:
            return bbox
        
    def __len__(self):
        return self.bbox_data.shape[0]

### model

In [ ]:
class LinearNet(nn.Module):
    
    def __init__(self, linear_num=2, feature_num=100):
        super(LinearNet, self).__init__()
        self.linear_num = linear_num
        self.feature_num = feature_num
        
        # define activation function
        self.relu = nn.ReLU(inplace=True)
        
        # create model
        layers = []
        for i in range(linear_num):
            if i == 0:
                linear = nn.Linear(3, feature_num)
                layers += [linear, self.relu]
            elif i == linear_num - 1:
                linear = nn.Linear(feature_num, 1)
                layers += [linear]
            else:
                linear = nn.Linear(feature_num, feature_num)
                layers += [linear, self.relu]
        
        self.model = nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.model(x)
        return x

### training

In [ ]:
def train(model, train_dataloader, test_dataloader, lr=1e-4, nepoch=100, visdom=False):
    # define loss
    l1_loss = nn.L1Loss(reduction='mean')
    # define optimizer
    opt = AdamW(model.parameters(), lr=lr)
    # set model train mode
    model.train()
    
    for epoch in range(nepoch):
        total_train_loss = 0
        total_test_loss = 0
        # train
        count = 0
        for bbox, size in train_dataloader:
            count += 1
            if args.cuda is True:
                bbox = bbox.cuda()
                size = size.cuda()
            opt.zero_grad()
            out = model(bbox)
            train_loss = l1_loss(out, size[:, None])
            total_train_loss += train_loss.item()
            train_loss.backward()
            opt.step()
        
        total_train_avg_loss = total_train_loss / count
        
        # valid
        model.eval()
        count = 0
        for bbox, size in test_dataloader:
            count += 1
            if args.cuda is True:
                bbox = bbox.cuda()
                size = size.cuda()
            out = model(bbox)
            test_loss = l1_loss(out, size[:, None])
            total_test_loss += test_loss.item()
            
        total_test_avg_loss = total_test_loss / count
        model.train()
        
        if visdom:
            visualize(vis, epoch+1, total_train_avg_loss, win_train_loss)
            visualize(vis, epoch+1, total_test_avg_loss, win_test_loss)
        print("epoch=%s: train_loss=%f, test_loss=%f, example={output: %f, target: %f}" % (epoch, total_train_avg_loss, total_test_avg_loss, out[0, 0], size[0]))

In [ ]:
# load data
bbox_df = pd.read_csv(args.bbox_data_path)
bbox_data = np.array(bbox_df.loc[:, ["width", "height", "label"]])
size_data = np.array(bbox_df.loc[:, "size"])

# define kfold
kf = KFold(n_splits=5)
valid_count = 0

# cross validation
for train_index, test_index in kf.split(bbox_data):
    print("")
    valid_count += 1
    print("----- valid {} -----".format(valid_count))
    print("")
    # create validation data
    bbox_train, bbox_test = bbox_data[train_index], bbox_data[test_index]
    size_train, size_test = size_data[train_index], size_data[test_index]
    # create dataloader
    train_dataset = bbox2size_dataset(bbox_train, size_train, training=True)
    train_dataloader = data.DataLoader(train_dataset, args.bs, num_workers=0, shuffle=True)
    test_dataset = bbox2size_dataset(bbox_test, size_test, training=False, evaluation=True)
    test_dataloader = data.DataLoader(test_dataset, 1, num_workers=0, shuffle=False)
    
    # create model
    model = LinearNet(linear_num=args.linear_num, feature_num=args.feature_num).cuda()
    
    # training
    train(model, train_dataloader, test_dataloader, lr=args.lr, nepoch=args.nepoch, visdom=args.visdom)
    torch.save(model.state_dict(), pj(args.model_root, "final.pth"))